In [1]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode
import codecs


meses = ['jan','fev','mar','abril','mai','jun','jul','ago','set','out','nov','dez']

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')
        
def date_format(date):
    return f"{date.day}/{meses[date.month-1]}"

In [2]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)

casos_excluidos = casos_confirmados.loc[casos_confirmados['mun_resid']=='EXCLUIR']
casos_confirmados = casos_confirmados.loc[casos_confirmados['mun_resid']!='EXCLUIR']

casos_confirmadosPR = casos_confirmados[casos_confirmados['rs'].notnull()]
print(f"Casos excluidos {len(casos_excluidos)}")
print(f"Casos confirmados PR {len(casos_confirmadosPR)}")
print(f"Casos confirmados Total {len(casos_confirmados)}")

Casos excluidos 3
Casos confirmados PR 243651
Casos confirmados Total 246166


In [3]:
obitos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Obitos', usecols='D:I')
obitos_confirmados.columns = [ normalize_labels(x) for x in obitos_confirmados.columns]
obitos_confirmados['nome'] = obitos_confirmados['nome'].apply(normalize_text)
obitos_confirmados['idade'] = obitos_confirmados['idade'].apply(normalize_number)
obitos_confirmados['municipio'] = obitos_confirmados['municipio'].apply(normalize_text)

obitos_excluidos = obitos_confirmados.loc[obitos_confirmados['municipio']=='EXCLUIR']
obitos_confirmados = obitos_confirmados.loc[obitos_confirmados['municipio']!='EXCLUIR']


obitos_confirmadosPR = obitos_confirmados[obitos_confirmados['rs'].notnull()]
print(f"Obitos excluidos {len(obitos_excluidos)}")
print(f"Obitos confirmados PR {len(obitos_confirmadosPR)}")
print(f"Obitos confirmados Total {len(obitos_confirmados)}")

Obitos excluidos 3
Obitos confirmados PR 5739
Obitos confirmados Total 5790


In [4]:
novos_casos = pd.read_excel('novos_casos.xlsx', dtype={'rs':str})
print(f"Total de novos casos: {len(novos_casos)}")
novos_obitos = pd.read_excel('novos_obitos.xlsx', dtype={'rs':str})
print(f"Total de novos obitos: {len(novos_obitos)}")
novos_casosPR = novos_casos.loc[novos_casos['rs'].notnull()]
print(f"Total de novos casos PR: {len(novos_casosPR)}")
novos_obitosPR = novos_obitos.loc[novos_obitos['rs'].notnull()]
print(f"Total de novos obitos PR: {len(novos_obitosPR)}")
novos_casosFora = novos_casos.loc[novos_casos['rs'].isnull()]
print(f"Total de novos casos Fora: {len(novos_casosFora)}")
novos_obitosFora = novos_obitos.loc[novos_obitos['rs'].isnull()]
print(f"Total de novos obitos Fora: {len(novos_obitosFora)}")

Total de novos casos: 2687
Total de novos obitos: 37
Total de novos casos PR: 2678
Total de novos obitos PR: 36
Total de novos casos Fora: 9
Total de novos obitos Fora: 1


In [5]:
novos_obitosPR_group = novos_obitosPR.groupby(by='mun_resid')
today = datetime.today().strftime('%d/%m/%Y - %Hh %M')
with codecs.open(f"relatorio_{(today.replace('/','_').replace(' ',''))}.txt","w","utf-8-sig") as relatorio:
    relatorio.write(f"{today}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))
    if len(novos_casosFora) > 0:
        relatorio.write(f"{len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"no PR.\n\n")
    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))
    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))
    if len(novos_obitosFora) > 0:
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))
    for index, row in novos_obitosPR.iterrows():
        relatorio.write(f"{row['sexo']} {row['idade']} {row['mun_resid']} {row['rs']} {row['data_cura_obito'].day}/{meses[row['data_cura_obito'].month-1]}\n")

In [6]:
# for grupo, valor in novos_casosPR.groupby(by='data_liberacao'):
#     print(f"{grupo}: {len(valor)}")